In [4]:
# # Install essential packages
# install.packages("tidyverse")
# install.packages("ggplot2")
# install.packages("dplyr")
# install.packages("caret")
# install.packages("randomForest")
# install.packages("e1071")

### Import Libraries

In [6]:
# Load essential libraries
library(tidyverse)
library(caret)
library(ggplot2)
library(randomForest)
library(e1071)

# Install and load the conflicted package (if not installed, install first)
if (!requireNamespace("conflicted", quietly = TRUE)) {
  install.packages("conflicted")
}
library(conflicted)

# Explicitly resolve conflicts
conflicted::conflict_prefer("filter", "dplyr")  # Always use dplyr::filter()
conflicted::conflict_prefer("lag", "dplyr")     # Always use dplyr::lag()


[conflicted] Removing existing preference.
[conflicted] Will prefer dplyr::filter over any other package.
[conflicted] Removing existing preference.
[conflicted] Will prefer dplyr::lag over any other package.


### Load Data set

In [3]:
# Get the current directory
getwd()

[1] "/home/jparep/proj/healthcare-analytics-with-R"

In [4]:
data <- read.csv("./data/US_chronic_disease_indicator.csv")
data

YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,⋯,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,⋯,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>
2015,2015,AK,Alaska,YRBSS,Alcohol,Alcohol use among youth,NA,%,Crude Prevalence,⋯,2,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NA,NA,NA,NA
2015,2015,AL,Alabama,YRBSS,Alcohol,Alcohol use among youth,NA,%,Crude Prevalence,⋯,1,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NA,NA,NA,NA
2015,2015,AR,Arkansas,YRBSS,Alcohol,Alcohol use among youth,NA,%,Crude Prevalence,⋯,5,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NA,NA,NA,NA
2015,2015,AZ,Arizona,YRBSS,Alcohol,Alcohol use among youth,NA,%,Crude Prevalence,⋯,4,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NA,NA,NA,NA
2015,2015,CA,California,YRBSS,Alcohol,Alcohol use among youth,NA,%,Crude Prevalence,⋯,6,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NA,NA,NA,NA
2015,2015,CO,Colorado,YRBSS,Alcohol,Alcohol use among youth,NA,%,Crude Prevalence,⋯,8,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NA,NA,NA,NA
2015,2015,CT,Connecticut,YRBSS,Alcohol,Alcohol use among youth,NA,%,Crude Prevalence,⋯,9,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NA,NA,NA,NA
2015,2015,DC,District of Columbia,YRBSS,Alcohol,Alcohol use among youth,NA,%,Crude Prevalence,⋯,11,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NA,NA,NA,NA
2015,2015,FL,Florida,YRBSS,Alcohol,Alcohol use among youth,NA,%,Crude Prevalence,⋯,12,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NA,NA,NA,NA
